## support vector machine con grid search

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics
import pandas as pd
from matplotlib import pyplot as plt
from statistics import mean
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


#setting up labels for dataset
labels = ['class', 'spec_num', 'eccentr', 'asp_ratio', 'elong', 'solidity', 'stoch_conv', 'iso_factor', 'max_ind_depth', 'lobedness', 'av_intensity', 'av_contr', 'smooth', 'third_mom', 'unif', 'entropy']
#importing data
df = pd.read_csv(r'./leaf/leaf.csv', header = None, names = labels)
# shuffling the dataframe
df = df.sample(frac=1).reset_index()
df = df.iloc[:, 1:17]   # needed to eliminate the old indexes column
#display(df)
#separating y from x
X = df.iloc[:, 2:16]
y = df.iloc[:, 0]

#display(X)

#### grid search con k-fold CV e LOOCV

In [ ]:
k = 5

# 'preparazione' indici di effectivness
# si usa la funzione make_scorer per costruire le metriche che ci servono
b_accuracy = metrics.make_scorer(metrics.balanced_accuracy_score)
recall = metrics.make_scorer(metrics.recall_score, average='weighted')
scorers = {'balanced_accuracy': b_accuracy, 'recall': recall}

# building the range of the regularization parameter (C)
exp = np.arange(-10, 12)
reg_param = 10.**exp

grid_param = {'C': reg_param,
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
              'degree': np.arange(2, 5),
              'decision_function_shape': ['ovo', 'ovr']}

clf_cv = GridSearchCV(svm.SVC(), grid_param, cv=k, scoring=scorers, refit=False)
clf_cv.fit(X, y)

In [ ]:
# showing the mean values of effectivness indexes

results_cv = pd.DataFrame(clf_cv.cv_results_)

display(results_cv.loc[:, ('params', 'mean_test_balanced_accuracy', 'rank_test_balanced_accuracy', 'mean_test_recall', 'rank_test_recall')])

qua sotto visualizziamo soltanto le righe con i valori più alti della balanced accuracy.


si hanno sei set di parametri che condividono la prima posizione perché:
1) il kernel migliore è quello lineare, pertanto param_degree è ininfluente rispetto alla costruzione del modello
2) per qualche motivo (da chiarire) ovo e ovr sono equivalenti


visti i buoni risultati per l'effectivness, potremmo fare una seconda grid search con kernel='linear' e C in un range attorno a pram_C

In [ ]:
display(results_cv[results_cv["rank_test_balanced_accuracy"]==1])

In [ ]:
# building the range of the regularization parameter (C)
exp = np.arange(-8, 8)
reg_param = 10.**exp

grid_param = {'C': reg_param,
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
              'degree': np.arange(2, 5),
              'decision_function_shape': ['ovo', 'ovr']}

clf_loocv = GridSearchCV(svm.SVC(), grid_param, cv=LeaveOneOut(), scoring='accuracy')
clf_loocv.fit(X, y)
print(clf_loocv.best_params_)
print(clf_loocv.best_score_)